In [44]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import shutil
import random
import numpy as np

In [7]:
class ImprovedTransferNetVGG19(models.Model):
    def __init__(self, base_model_weights=None):
        super(ImprovedTransferNetVGG19, self).__init__()
        self.base_model = tf.keras.applications.VGG19(include_top=False, weights=base_model_weights, input_shape=(224, 224, 3))
        for layer in self.base_model.layers:
            layer.trainable = False  # Freeze VGG19 layers initially
        self.flatten = layers.Flatten()
        
        l1lambda = 0.0001
        # Dense layers with reduced neurons
        self.fc1 = layers.Dense(1024, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(l1lambda))
        self.fc1_bn = layers.BatchNormalization()
        self.fc1_dropout = layers.Dropout(0.4)

        self.fc2 = layers.Dense(512, activation='relu')
        self.fc2_bn = layers.BatchNormalization()
        self.fc2_dropout = layers.Dropout(0.4)
        
        self.fc3 = layers.Dense(128, activation='relu')
        self.fc3_bn = layers.BatchNormalization()
        self.fc3_dropout = layers.Dropout(0.4)

        self.fc4 = layers.Dense(4, activation='softmax')

    def call(self, x):
        x = self.base_model(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc1_bn(x)
        x = self.fc1_dropout(x)
        
        x = self.fc2(x)
        x = self.fc2_bn(x)
        x = self.fc2_dropout(x)

        x = self.fc3(x)
        x = self.fc3_bn(x)
        x = self.fc3_dropout(x)

        x = self.fc4(x)
        return x
    def unfreeze_last_layers(self, num_layers=6):
        """Desbloquea las últimas num_layers del modelo base."""
        for layer in self.base_model.layers[-num_layers:]:
            layer.trainable = True

    def get_config(self):
        return {"base_model_name": "VGG19"}

    @classmethod
    def from_config(cls, config):
        if config["base_model_name"] == "VGG19":
            return cls(base_model_weights="imagenet")


In [27]:
def split_data(dataset_dir, train_dir, val_dir, test_dir, split_ratio=(0.8, 0.1, 0.1)):
    # If the directory already exists, delete it
    for directory in [train_dir, val_dir, test_dir]:
        if os.path.exists(directory):
            shutil.rmtree(directory)

    # Create the directory if it does not exist
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Collect the filenames
    image_files = os.listdir(dataset_dir)
    random.shuffle(image_files)

    # Split the data
    total_images = len(image_files)
    num_train = int(total_images * split_ratio[0])
    num_val = int(total_images * split_ratio[1])
    num_test = total_images - num_train - num_val

    # Copy the images to the correct directory
    for i, image_file in enumerate(image_files):
        src_path = os.path.join(dataset_dir, image_file)
        if i < num_train:
            dst_path = os.path.join(train_dir, image_file)
        elif i < num_train + num_val:
            dst_path = os.path.join(val_dir, image_file)
        else:
            dst_path = os.path.join(test_dir, image_file)
        shutil.copy(src_path, dst_path)

    print("Splitting data completed.")

In [30]:
# Define paths for the images
downdog_SRC_DIR = "FINALDATA/downdog"
plank_SRC_DIR = "FINALDATA/plank"
tree_SRC_DIR = "FINALDATA/tree"
warrior_SRC_DIR = "FINALDATA/warrior"
TRAINING_DIR = "dataset/training"
VALIDATION_DIR = "dataset/validation"
TEST_DIR = "dataset/test"
#splitting the dataset
TRAINING_DDG_DIR = os.path.join(TRAINING_DIR, "downdog/")
VALIDATION_DDG_DIR = os.path.join(VALIDATION_DIR, "downdog/")
TEST_DDG_DIR = os.path.join(TEST_DIR, "downdog/")

TRAINING_PLANK_DIR = os.path.join(TRAINING_DIR, "plank/")
VALIDATION_PLANK_DIR = os.path.join(VALIDATION_DIR, "plank/")
TEST_PLANK_DIR = os.path.join(TEST_DIR, "plank/")

TRAINING_TREE_DIR = os.path.join(TRAINING_DIR, "tree/")
VALIDATION_TREE_DIR = os.path.join(VALIDATION_DIR, "tree/")
TEST_TREE_DIR = os.path.join(TEST_DIR, "tree/")

TRAINING_WR_DIR = os.path.join(TRAINING_DIR, "warrior/")
VALIDATION_WR_DIR = os.path.join(VALIDATION_DIR, "warrior/")
TEST_WR_DIR = os.path.join(TEST_DIR, "warrior/")
# Define proportion of images used for training
train_size = 0.8
val_size = 0.1
test_size = 0.1

split_data(downdog_SRC_DIR,
           TRAINING_DDG_DIR,
           VALIDATION_DDG_DIR,
           TEST_DDG_DIR,
           split_ratio=(train_size,val_size, test_size))

split_data(plank_SRC_DIR,
           TRAINING_PLANK_DIR,
           VALIDATION_PLANK_DIR,
           TEST_PLANK_DIR,
           split_ratio=(train_size,val_size, test_size))

split_data(tree_SRC_DIR,
           TRAINING_TREE_DIR,
           VALIDATION_TREE_DIR,
           TEST_TREE_DIR,
           split_ratio=(train_size,val_size, test_size))
split_data(warrior_SRC_DIR,
           TRAINING_WR_DIR,
           VALIDATION_WR_DIR,
           TEST_WR_DIR,
           split_ratio=(train_size,val_size, test_size))

Splitting data completed.
Splitting data completed.
Splitting data completed.
Splitting data completed.


In [31]:
# Definir un generador de imágenes con transformaciones
batch_size = 32
width=224
height=224
posture_names = ["downdog", "plank", "tree", "warrior"]
data_generator = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.6,1.0],
    validation_split=0.20
)

train_data = data_generator.flow_from_directory(
    TRAINING_DIR,
    classes = posture_names,
    target_size=(width, height),
    batch_size=batch_size,
    class_mode='sparse',  
    subset='training',
    color_mode="rgb"
)

validation_data = data_generator.flow_from_directory(
    VALIDATION_DIR,
    classes = posture_names,
    target_size=(width, height),
    batch_size=batch_size,
    color_mode="rgb",
    class_mode='sparse',
    subset='validation',
)



Found 1769 images belonging to 4 classes.
Found 55 images belonging to 4 classes.


In [4]:
def create_model_instance():
    return ImprovedTransferNetVGG19(base_model_weights=None)  # Aquí no cargamos los pesos preentrenados de VGG19 ya que vamos a cargar tus pesos guardados


In [16]:
model_instance = create_model_instance()
model_instance.build((None, 224, 224, 3))  # Esto es necesario para construir el modelo antes de cargar los pesos
model_instance.load_weights('v5.h5')

ValueError: Cannot assign value to variable ' block1_conv1/kernel:0': Shape mismatch.The variable shape (3, 3, 3, 64), and the assigned value shape (512, 512, 3, 3) are incompatible.

In [32]:
model_files = ['v2.h5',  'v4.h5']
evaluation_results = {}

for model_file in model_files:
    model_instance = create_model_instance()
    model_instance.build((None, 224, 224, 3))  # Esto es necesario para construir el modelo antes de cargar los pesos
    model_instance.load_weights(model_file)
    
    optimizer = tf.keras.optimizers.Adam()  # Puedes usar el optimizador que prefieras
    model_instance.compile(optimizer=optimizer, 
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                           metrics=['accuracy'])
    
    loss, accuracy = model_instance.evaluate(validation_data, validation_labels)
    evaluation_results[model_file] = (loss, accuracy)


NameError: name 'validation_labels' is not defined

In [ ]:
best_model_name = max(evaluation_results, key=lambda x: evaluation_results[x][1])  # Obtenemos el modelo con la máxima precisión
print(f"El mejor modelo es: {best_model_name}")



In [ ]:
best_model_name = max(evaluation_results, key=evaluation_results.get)
print(f"El mejor modelo es: {best_model_name}")


In [13]:
# Lista de tus archivos .h5
model_files = ["v1.h5", "v2.h5", "v3.h5", "v4.h5"]

# Función para evaluar un modelo
def evaluate_model(model_file, is_finetuned=False):
    # Crear una instancia de tu modelo
    model = ImprovedTransferNetVGG19(base_model_weights=None)
    
    # Si el modelo fue fine-tuned, descongela las capas adecuadas
    if is_finetuned:
        model.unfreeze_last_layers(num_layers=6)
    
    # Compilar el modelo (puedes ajustar estos parámetros según lo necesites)
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    model.compile(optimizer=optimizer, 
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                  metrics=['accuracy'])
    
    # Cargar los pesos
    model.load_weights(model_file)
    
    # Evaluar el modelo con tu conjunto de validación/test
    loss, accuracy = model.evaluate(validation_data)
    
    return loss, accuracy

# Evalúa cada modelo y almacena sus resultados
results = {}
for model_file in model_files:
    is_finetuned = determine_if_finetuned(model_file)  # Función que necesitas crear para saber si el modelo fue fine-tuned
    loss, accuracy = evaluate_model(model_file, is_finetuned)
    results[model_file] = (loss, accuracy)

# Imprime los resultados o decide qué modelo es el mejor según tus criterios
print(results)


NameError: name 'determine_if_finetuned' is not defined

In [35]:
def create_and_load_model(model_file):
    # Crear una instancia del modelo
    model_instance = ImprovedTransferNetVGG19(base_model_weights=None)
    model_instance.build((None, 224, 224, 3))  # Construir el modelo
    
    # Intenta cargar los pesos
    try:
        model_instance.load_weights(model_file)
        print(f"{model_file} cargado correctamente.")
    except Exception as e:
        # Si hay un error, asumimos que es un modelo fine-tuned
        model_instance.unfreeze_last_layers(num_layers=6)
        model_instance.load_weights(model_file)
        print(f"{model_file} (fine-tuned) cargado correctamente.")
    
    return model_instance

# Compilar y evaluar los modelos
results = {}
for model_file in ["v1.h5", "v2.h5", "v3.h5", "v4.h5","v5.h5"]:
    model = create_and_load_model(model_file)
    
    # Compilar el modelo
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5)
    model.compile(optimizer=optimizer, 
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                  metrics=['accuracy'])
    
    # Evaluar el modelo con tu conjunto de validación/test
    loss, accuracy = model.evaluate(validation_data)
    
    results[model_file] = (loss, accuracy)

# Imprime los resultados
print(results)


v1.h5 (fine-tuned) cargado correctamente.


2023-08-23 20:38:18.934522: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 4s 1s/step - loss: 4.9335 - accuracy: 0.5636
v2.h5 cargado correctamente.


2023-08-23 20:38:23.334663: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 4s 1s/step - loss: 0.6859 - accuracy: 0.7818
v3.h5 (fine-tuned) cargado correctamente.


...

2/2 [==============================] - 3s 1s/step - loss: 15.8196 - accuracy: 0.4364
v4.h5 cargado correctamente.


2023-08-23 20:38:32.633610: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 4s 1s/step - loss: 17.0028 - accuracy: 0.4000
v5.h5 (fine-tuned) cargado correctamente.


2023-08-23 20:38:36.840514: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 3s 1s/step - loss: 0.4870 - accuracy: 0.8182
{'v1.h5': (4.933504581451416, 0.5636363625526428), 'v2.h5': (0.6859138011932373, 0.7818182110786438), 'v3.h5': (15.819561004638672, 0.4363636374473572), 'v4.h5': (17.002777099609375, 0.4000000059604645), 'v5.h5': (0.4870135188102722, 0.8181818127632141)}


In [42]:
len(validation_data)
x_val, y_val = validation_data  # Si es una lista o tupla


ValueError: too many values to unpack (expected 2)

In [45]:
from sklearn.metrics import recall_score, f1_score, confusion_matrix
def extract_data_from_generator(generator):
    x_val_list = []
    y_val_list = []

    for i in range(len(generator)):
        x_batch, y_batch = generator[i]
        x_val_list.extend(x_batch)
        y_val_list.extend(y_batch)

    return np.array(x_val_list), np.array(y_val_list)

x_val, y_val = extract_data_from_generator(validation_data)

# Función para evaluar un modelo
def evaluate_model(filename, model_instance, x_val, y_val):
    model_instance.load_weights(filename)
    y_pred = np.argmax(model_instance.predict(x_val), axis=1)
    
    recall = recall_score(y_val, y_pred, average='macro') # 'macro' calcula métricas para cada etiqueta y encuentra su media no ponderada
    f1 = f1_score(y_val, y_pred, average='macro')
    conf_matrix = confusion_matrix(y_val, y_pred)
    
    return recall, f1, conf_matrix
# Evaluar cada modelo
models = ["v1.h5", "v2.h5", "v3.h5", "v4.h5", "v5.h5"]
metrics = {}

for model in models:
    recall, f1, conf_matrix = evaluate_model(model, model_instance, x_val, y_val)
    metrics[model] = {"recall": recall, "f1": f1, "confusion_matrix": conf_matrix}

# Mostrar resultados
for model, values in metrics.items():
    print(f"Model {model}:")
    print(f"Recall: {values['recall']}")
    print(f"F1-score: {values['f1']}")
    print(f"Confusion Matrix:\n{values['confusion_matrix']}\n")


ValueError: Cannot assign value to variable ' block1_conv1/kernel:0': Shape mismatch.The variable shape (3, 3, 3, 64), and the assigned value shape (512, 512, 3, 3) are incompatible.

In [46]:
from sklearn.metrics import recall_score, f1_score, confusion_matrix

def extract_data_from_generator(generator):
    x_val_list = []
    y_val_list = []

    for i in range(len(generator)):
        x_batch, y_batch = generator[i]
        x_val_list.extend(x_batch)
        y_val_list.extend(y_batch)

    return np.array(x_val_list), np.array(y_val_list)

x_val, y_val = extract_data_from_generator(validation_data)

# Función para evaluar un modelo
def evaluate_model(filename):
    model_instance = create_and_load_model(filename)
    y_pred = np.argmax(model_instance.predict(x_val), axis=1)
    
    recall = recall_score(y_val, y_pred, average='macro') # 'macro' calcula métricas para cada etiqueta y encuentra su media no ponderada
    f1 = f1_score(y_val, y_pred, average='macro')
    conf_matrix = confusion_matrix(y_val, y_pred)
    
    return recall, f1, conf_matrix

# Evaluar cada modelo
models = ["v1.h5", "v2.h5", "v3.h5", "v4.h5", "v5.h5"]
metrics = {}

for model in models:
    recall, f1, conf_matrix = evaluate_model(model)
    metrics[model] = {"recall": recall, "f1": f1, "confusion_matrix": conf_matrix}

# Mostrar resultados
for model, values in metrics.items():
    print(f"Model {model}:")
    print(f"Recall: {values['recall']}")
    print(f"F1-score: {values['f1']}")
    print(f"Confusion Matrix:\n{values['confusion_matrix']}\n")


v1.h5 (fine-tuned) cargado correctamente.


2023-08-23 20:52:33.552357: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 2s 2s/step
v2.h5 cargado correctamente.


2023-08-23 20:52:37.226437: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 2s 1s/step
v3.h5 (fine-tuned) cargado correctamente.


...

2/2 [==============================] - 2s 1s/step
v4.h5 cargado correctamente.


2023-08-23 20:52:43.565859: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 2s 1s/step
v5.h5 (fine-tuned) cargado correctamente.


2023-08-23 20:52:46.696913: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 2s 1s/step
Model v1.h5:
Recall: 0.48351648351648346
F1-score: 0.37962962962962965
Confusion Matrix:
[[14  0  0  0]
 [14  0  0  0]
 [10  0  1  2]
 [ 2  0  0 12]]

Model v2.h5:
Recall: 0.8186813186813188
F1-score: 0.8158195970695971
Confusion Matrix:
[[14  0  0  0]
 [ 4 10  0  0]
 [ 0  0 11  2]
 [ 0  0  4 10]]

Model v3.h5:
Recall: 0.4642857142857143
F1-score: 0.39086956521739136
Confusion Matrix:
[[ 0  3  8  3]
 [ 0  6  8  0]
 [ 0  0 13  0]
 [ 0  0  8  6]]

Model v4.h5:
Recall: 0.42857142857142855
F1-score: 0.3348684210526316
Confusion Matrix:
[[ 9  0  5  0]
 [ 1  0 13  0]
 [ 0  0 13  0]
 [ 0  0 13  1]]

Model v5.h5:
Recall: 0.8159340659340659
F1-score: 0.8131029793021378
Confusion Matrix:
[[14  0  0  0]
 [ 5  9  0  0]
 [ 0  0  9  4]
 [ 0  0  1 13]]

